# ET Code

---

In [2]:
%%capture

!pip install faker
!pip install python-dateutil

---

### Creating sub-directories for data

In [3]:
# create sub-directories
!mkdir ../data/test
!mkdir ../data/train
!mkdir ../data/transformed
!mkdir ../data/partitions
!mkdir ../data/validation

In [4]:
# Other imports
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from datetime import datetime, timezone, date
from faker import Faker
import pandas as pd
import numpy as np
import hashlib
import logging
import random

In [5]:
# Make a Faker var, and seed it with locale and rand_seed
faker = Faker()
faker.seed_locale('en_US', 0)

In [6]:
SEED = 123
random.seed(SEED)
np.random.seed(SEED)
faker.seed_instance(SEED)

In [8]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [9]:
logger.info(f'Using Pandas version: {pd.__version__}')

Using Pandas version: 1.3.5


---

## Helper Functions

In [10]:
def generate_timestamp(start, end) -> str:
    start = datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
    end = datetime.strptime(end, '%Y-%m-%d %H:%M:%S')
    timestamp = faker.date_time_between(start_date=start, end_date=end, tzinfo=None).strftime('%Y-%m-%d %H:%M:%S')
    return timestamp

In [11]:
def generate_date(start, end) -> str:
    start = datetime.strptime(start, '%Y-%m-%d')
    end = datetime.strptime(end, '%Y-%m-%d')
    date = faker.date_between_dates(date_start=start, date_end=end).strftime('%Y-%m-%d')
    return date

In [13]:
def get_md5_hash(string: str) -> str:
    hash_object = hashlib.md5(string.encode())
    return hash_object.hexdigest()

In [14]:
def generate_event_timestamp():
    naive_dt = datetime.now()
    aware_dt = naive_dt.astimezone()
    utc_dt = aware_dt.astimezone(timezone.utc)
    event_time = utc_dt.isoformat(timespec='milliseconds')
    event_time = event_time.replace('+00:00', 'Z')
    return event_time

---

## Generate Online Grocery Shopping Dataset

In [15]:
class Customer:
    def __init__(self):
        self.customer_id = None
        self.name = None
        self.sex = None
        self.state = None
        self.age = None
        self.is_married = None
        self.active_since = None
        self.event_time = None
        
    def as_dict(self):
        return {'customer_id': self.customer_id, 
                'name': self.name,
                'sex': self.sex, 
                'state': self.state, 
                'age': self.age, 
                'is_married': self.is_married, 
                'active_since': self.active_since,
                'event_time': self.event_time
               }

In [19]:
def generate_customer(i) -> Customer:
    cust = Customer()
    profile = faker.profile()
    cust.customer_id = f'C{i}'
    cust.name = profile['name'].lower()
    cust.sex = profile['sex']
    cust.state = faker.state().lower()
    cust.age = random.randint(18,91)
    cust.is_married = faker.boolean()
    cust.active_since = generate_timestamp('2016-01-01 00:00:00', '2020-01-01 00:00:01')
    cust.event_time = generate_event_timestamp()
    return cust

In [23]:
customer = generate_customer(1)
customer.__dict__

{'customer_id': 'C1',
 'name': 'eric campbell',
 'sex': 'M',
 'state': 'missouri',
 'age': 52,
 'is_married': True,
 'active_since': '2016-06-13 05:47:10',
 'event_time': '2023-05-22T12:02:18.253Z'}

In [25]:
customers = []
customer_ids = []
n = 10000
for i in range(n):
    customer = generate_customer(i+1)
    customers.append(customer)
    customer_ids.append(customer.customer_id)

In [26]:
customers_df = pd.DataFrame([customer.as_dict() for customer in customers])

In [27]:
customers_df.head(5)

,customer_id,name,sex,state,age,is_married,active_since,event_time
0,C1,stephanie jones,F,new hampshire,22,True,2019-06-24 02:57:56,2023-05-22T12:03:46.119Z
1,C2,todd jones,M,colorado,66,True,2016-12-01 12:47:02,2023-05-22T12:03:46.122Z
2,C3,janet taylor,F,arkansas,86,False,2018-03-12 05:24:32,2023-05-22T12:03:46.126Z
3,C4,ryan bailey,M,vermont,89,False,2019-02-19 01:27:02,2023-05-22T12:03:46.129Z
4,C5,larry martinez,M,hawaii,60,True,2017-12-21 02:23:15,2023-05-22T12:03:46.135Z


In [28]:
products_df = pd.read_csv("../data/raw/product_category_mapping.csv")
products_df['product_name'] = products_df['product_name'].str.lower()
products_df['product_category'] = products_df['product_category'].str.lower()
products_df.head(5)

,product_name,product_category,product_id
0,chocolate sandwich cookies,cookies cakes,P1
1,all-seasons salt,spices seasonings,P2
2,robust golden unsweetened oolong tea,tea,P3
3,smart ones classic favorites mini rigatoni wit...,frozen meals,P4
4,pure coconut water with orange,juice nectars,P5


In [29]:
event_timestamps = [generate_event_timestamp() for _ in range(len(products_df))]
products_df['event_time'] = event_timestamps
products_df.head(5)

,product_name,product_category,product_id,event_time
0,chocolate sandwich cookies,cookies cakes,P1,2023-05-22T12:05:37.598Z
1,all-seasons salt,spices seasonings,P2,2023-05-22T12:05:37.599Z
2,robust golden unsweetened oolong tea,tea,P3,2023-05-22T12:05:37.599Z
3,smart ones classic favorites mini rigatoni wit...,frozen meals,P4,2023-05-22T12:05:37.599Z
4,pure coconut water with orange,juice nectars,P5,2023-05-22T12:05:37.599Z


In [30]:
product_ids = products_df['product_id'].tolist()

In [32]:
class Order:
    def __init__(self):
        self.order_id = None
        self.customer_id = None
        self.product_id = None
        self.purchase_amount = None
        self.is_reordered = None 
        self.purchased_on = None
        self.event_time = None
        
    def as_dict(self):
        return {'order_id': self.order_id, 
                'customer_id': self.customer_id, 
                'product_id': self.product_id,
                'purchase_amount': self.purchase_amount,
                'is_reordered': self.is_reordered,
                'purchased_on': self.purchased_on, 
                'event_time': self.event_time}

In [33]:
def generate_order(i) -> Order:
    order = Order()
    order.order_id = f'O{i}'
    order.customer_id = random.choice(customer_ids)
    order.product_id = random.choice(product_ids)
    order.purchase_amount = random.randint(1, 101) + round(random.random(), 2)
    order.is_reordered = random.choice([1, 1, 0])  # assume chance of reordering is twice as that of not reordering
    order.purchased_on = generate_timestamp('2020-01-01 00:01:01', '2021-06-01 00:00:01')
    order.event_time = generate_event_timestamp()
    return order

In [34]:
order = generate_order(1)
order.__dict__

{'order_id': 'O1',
 'customer_id': 'C5731',
 'product_id': 'P16',
 'purchase_amount': 93.26,
 'is_reordered': 1,
 'purchased_on': '2020-07-25 01:30:37',
 'event_time': '2023-05-22T12:07:15.872Z'}

In [35]:
orders = []
n = 100000  # number of synthetic orders to generate
for i in range(n):
    order = generate_order(i+1)
    orders.append(order)

In [36]:
orders_df = pd.DataFrame([order.as_dict() for order in orders])
orders_df.head(5)

,order_id,customer_id,product_id,purchase_amount,is_reordered,purchased_on,event_time
0,O1,C3541,P12802,67.98,1,2020-09-18 17:51:52,2023-05-22T12:07:44.825Z
1,O2,C7402,P8320,64.59,1,2020-07-09 16:34:01,2023-05-22T12:07:44.825Z
2,O3,C7356,P5165,63.51,0,2020-09-01 17:50:56,2023-05-22T12:07:44.825Z
3,O4,C5806,P12940,6.37,1,2020-11-07 15:06:17,2023-05-22T12:07:44.825Z
4,O5,C1570,P8025,57.88,1,2021-04-10 02:23:05,2023-05-22T12:07:44.825Z


In [37]:
orders_df.shape

(100000, 7)

In [39]:
customers_df.to_csv('../data/raw/customers.csv', index=False)

In [40]:
products_df.to_csv('../data/raw/products.csv', index=False)

In [41]:
orders_df.to_csv('../data/raw/orders.csv', index=False)

In [42]:
customers_df = pd.read_csv('../data/raw/customers.csv')
customers_df.head(5)

,customer_id,name,sex,state,age,is_married,active_since,event_time
0,C1,stephanie jones,F,new hampshire,22,True,2019-06-24 02:57:56,2023-05-22T12:03:46.119Z
1,C2,todd jones,M,colorado,66,True,2016-12-01 12:47:02,2023-05-22T12:03:46.122Z
2,C3,janet taylor,F,arkansas,86,False,2018-03-12 05:24:32,2023-05-22T12:03:46.126Z
3,C4,ryan bailey,M,vermont,89,False,2019-02-19 01:27:02,2023-05-22T12:03:46.129Z
4,C5,larry martinez,M,hawaii,60,True,2017-12-21 02:23:15,2023-05-22T12:03:46.135Z


In [43]:
label_encoder = LabelEncoder()
min_max_scaler = MinMaxScaler()

In [44]:
customers_df.drop('name', axis=1, inplace=True)
customers_df.drop('state', axis=1, inplace=True)

In [45]:
bins = [18, 30, 40, 50, 60, 70, 90]
labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70-plus']
customers_df['age_range'] = pd.cut(customers_df.age, bins, labels=labels, include_lowest=True)
customers_df = pd.concat([customers_df, pd.get_dummies(customers_df['age_range'], prefix='age')], axis=1)
customers_df.drop('age', axis=1, inplace=True)
customers_df.drop('age_range', axis=1, inplace=True)

In [46]:
customers_df['sex'] = label_encoder.fit_transform(customers_df['sex'])
customers_df['is_married'] = label_encoder.fit_transform(customers_df['is_married'])

In [47]:
customers_df.head()

,customer_id,sex,is_married,active_since,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus
0,C1,0,1,2019-06-24 02:57:56,2023-05-22T12:03:46.119Z,1,0,0,0,0,0
1,C2,1,1,2016-12-01 12:47:02,2023-05-22T12:03:46.122Z,0,0,0,0,1,0
2,C3,0,0,2018-03-12 05:24:32,2023-05-22T12:03:46.126Z,0,0,0,0,0,1
3,C4,1,0,2019-02-19 01:27:02,2023-05-22T12:03:46.129Z,0,0,0,0,0,1
4,C5,1,1,2017-12-21 02:23:15,2023-05-22T12:03:46.135Z,0,0,0,1,0,0


In [48]:
customers_df['active_since'] =  pd.to_datetime(customers_df['active_since'], format='%Y-%m-%d %H:%M:%S')

In [49]:
def get_delta_in_days(date_time) -> int:
    today = date.today()
    delta = today - date_time.date()
    return delta.days

In [50]:
customers_df['n_days_active'] = customers_df['active_since'].apply(lambda x: get_delta_in_days(x))
customers_df['n_days_active'] = min_max_scaler.fit_transform(customers_df[['n_days_active']])
customers_df.drop('active_since', axis=1, inplace=True)

In [51]:
customers_df.head()

,customer_id,sex,is_married,event_time,age_18-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-plus,n_days_active
0,C1,0,1,2023-05-22T12:03:46.119Z,1,0,0,0,0,0,0.130137
1,C2,1,1,2023-05-22T12:03:46.122Z,0,0,0,0,1,0,0.770548
2,C3,0,0,2023-05-22T12:03:46.126Z,0,0,0,0,0,1,0.451370
3,C4,1,0,2023-05-22T12:03:46.129Z,0,0,0,0,0,1,0.215753
4,C5,1,1,2023-05-22T12:03:46.135Z,0,0,0,1,0,0,0.506849


In [52]:
customers_df.to_csv('../data/transformed/customers.csv', index=False)

In [53]:
products_df = pd.read_csv('../data/raw/products.csv')
products_df.head(5)

,product_name,product_category,product_id,event_time
0,chocolate sandwich cookies,cookies cakes,P1,2023-05-22T12:05:37.598Z
1,all-seasons salt,spices seasonings,P2,2023-05-22T12:05:37.599Z
2,robust golden unsweetened oolong tea,tea,P3,2023-05-22T12:05:37.599Z
3,smart ones classic favorites mini rigatoni wit...,frozen meals,P4,2023-05-22T12:05:37.599Z
4,pure coconut water with orange,juice nectars,P5,2023-05-22T12:05:37.599Z


In [54]:
products_df.drop('product_name', axis=1, inplace=True)

In [55]:
products_df = pd.concat([products_df, pd.get_dummies(products_df['product_category'], prefix='category')], axis=1)

In [56]:
products_df.drop('product_category', axis=1, inplace=True)
products_df.columns = products_df.columns.str.replace(' ', '_')

In [57]:
products_df.head(5)

,product_id,event_time,category_baby_food_formula,category_baking_ingredients,category_candy_chocolate,category_chips_pretzels,category_cleaning_products,category_coffee,category_cookies_cakes,category_crackers,...,category_hair_care,category_ice_cream_ice,category_juice_nectars,category_packaged_cheese,category_refrigerated,category_soup_broth_bouillon,category_spices_seasonings,category_tea,category_vitamins_supplements,category_yogurt
0,P1,2023-05-22T12:05:37.598Z,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,P2,2023-05-22T12:05:37.599Z,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,P3,2023-05-22T12:05:37.599Z,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,P4,2023-05-22T12:05:37.599Z,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,P5,2023-05-22T12:05:37.599Z,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [58]:
products_df.to_csv('../data/transformed/products.csv', index=False)

In [59]:
orders_df = pd.read_csv('../data/raw/orders.csv')
orders_df.head(5)

,order_id,customer_id,product_id,purchase_amount,is_reordered,purchased_on,event_time
0,O1,C3541,P12802,67.98,1,2020-09-18 17:51:52,2023-05-22T12:07:44.825Z
1,O2,C7402,P8320,64.59,1,2020-07-09 16:34:01,2023-05-22T12:07:44.825Z
2,O3,C7356,P5165,63.51,0,2020-09-01 17:50:56,2023-05-22T12:07:44.825Z
3,O4,C5806,P12940,6.37,1,2020-11-07 15:06:17,2023-05-22T12:07:44.825Z
4,O5,C1570,P8025,57.88,1,2021-04-10 02:23:05,2023-05-22T12:07:44.825Z


In [60]:
orders_df['purchased_on'] =  pd.to_datetime(orders_df['purchased_on'], format='%Y-%m-%d %H:%M:%S')
orders_df['n_days_since_last_purchase'] = orders_df['purchased_on'].apply(lambda x: get_delta_in_days(x))
orders_df['n_days_since_last_purchase'] = min_max_scaler.fit_transform(orders_df[['n_days_since_last_purchase']])


In [61]:
orders_df['purchase_amount'] = min_max_scaler.fit_transform(orders_df[['purchase_amount']])
orders_df['is_reordered'] = label_encoder.fit_transform(orders_df['is_reordered'])

In [62]:
orders_df.head(5)

,order_id,customer_id,product_id,purchase_amount,is_reordered,purchased_on,event_time,n_days_since_last_purchase
0,O1,C3541,P12802,0.663168,1,2020-09-18 17:51:52,2023-05-22T12:07:44.825Z,0.494186
1,O2,C7402,P8320,0.629604,1,2020-07-09 16:34:01,2023-05-22T12:07:44.825Z,0.631783
2,O3,C7356,P5165,0.618911,0,2020-09-01 17:50:56,2023-05-22T12:07:44.825Z,0.527132
3,O4,C5806,P12940,0.053168,1,2020-11-07 15:06:17,2023-05-22T12:07:44.825Z,0.397287
4,O5,C1570,P8025,0.563168,1,2021-04-10 02:23:05,2023-05-22T12:07:44.825Z,0.098837


In [63]:
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta

print(f'Total Orders Count = {orders_df.shape[0]}') 
partitions_path = '../data/partitions'
start_date_str = '2020-01-01 00:00:00'
end_date_str = '2021-06-01 00:00:01'
date_format = '%Y-%m-%d %H:%M:%S'
start_date = datetime.strptime(start_date_str, date_format)
print(f'start_date = {start_date}')
end_date = datetime.strptime(end_date_str, date_format)
print(f'end_date = {end_date}')
a_month = relativedelta(months=1)
print(f'a_month = {a_month}')
current_start_date = start_date
print(f'current_start_date = {current_start_date}')
current_end_date = start_date + a_month
print(f'current_end_date = {current_end_date}')
print(f'----')
if not os.path.exists(partitions_path):
    os.makedirs(partitions_path)

while current_end_date <= end_date:
    print(f'Dates between {current_start_date} and {current_end_date}')
    partitions_df = orders_df[orders_df['purchased_on'].between(current_start_date, current_end_date)].copy()
    partitions_df.drop('purchased_on', axis=1, inplace=True)
    partition = f'{current_start_date.strftime("%Y")}-{int(current_start_date.strftime("%m"))}'    
    current_partitions_path = f'{partitions_path}/{partition}'
    print(current_partitions_path)
    if not os.path.exists(current_partitions_path):
        os.makedirs(current_partitions_path)
    print(f'Partitions Orders Count = {partitions_df.shape[0]}')
    partitions_df.to_csv(f'{current_partitions_path}/partition.csv', index=False)
    partitions_df.iloc[0:0]
    current_start_date = current_end_date
    current_end_date = current_start_date + a_month
    print(f'----')

Total Orders Count = 100000
start_date = 2020-01-01 00:00:00
end_date = 2021-06-01 00:00:01
a_month = relativedelta(months=+1)
current_start_date = 2020-01-01 00:00:00
current_end_date = 2020-02-01 00:00:00
----
Dates between 2020-01-01 00:00:00 and 2020-02-01 00:00:00
../data/partitions/2020-1
Partitions Orders Count = 6023
----
Dates between 2020-02-01 00:00:00 and 2020-03-01 00:00:00
../data/partitions/2020-2
Partitions Orders Count = 5508
----
Dates between 2020-03-01 00:00:00 and 2020-04-01 00:00:00
../data/partitions/2020-3
Partitions Orders Count = 5948
----
Dates between 2020-04-01 00:00:00 and 2020-05-01 00:00:00
../data/partitions/2020-4
Partitions Orders Count = 5873
----
Dates between 2020-05-01 00:00:00 and 2020-06-01 00:00:00
../data/partitions/2020-5
Partitions Orders Count = 5993
----
Dates between 2020-06-01 00:00:00 and 2020-07-01 00:00:00
../data/partitions/2020-6
Partitions Orders Count = 5795
----
Dates between 2020-07-01 00:00:00 and 2020-08-01 00:00:00
../data/pa

In [64]:
orders_df.drop('purchased_on', axis=1, inplace=True)

In [65]:
orders_df.head(5)

,order_id,customer_id,product_id,purchase_amount,is_reordered,event_time,n_days_since_last_purchase
0,O1,C3541,P12802,0.663168,1,2023-05-22T12:07:44.825Z,0.494186
1,O2,C7402,P8320,0.629604,1,2023-05-22T12:07:44.825Z,0.631783
2,O3,C7356,P5165,0.618911,0,2023-05-22T12:07:44.825Z,0.527132
3,O4,C5806,P12940,0.053168,1,2023-05-22T12:07:44.825Z,0.397287
4,O5,C1570,P8025,0.563168,1,2023-05-22T12:07:44.825Z,0.098837


In [66]:
orders_df.to_csv('../data/transformed/orders.csv', index=False)